## Perturbations

In [1]:
import torch
from perturbations import perturbed

dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DEPTH = 4
DIM = 2**DEPTH - 1

path_inds = torch.zeros(DEPTH, DIM).long()
path_inds[0] = torch.arange(start=0, end=DIM).long()
path_inds[1] = ((path_inds[0]-1)/2).long()
path_inds[2] = ((path_inds[1]-1)/2).long()
path_inds[3] = ((path_inds[2]-1)/2).long()

def best_path(inputs):
    """Returns the ranks of the input values among the given axis."""
    scores = inputs[..., path_inds].sum(dim=-2).squeeze(dim=-2)
    best_scores_ind = scores[..., int(DIM/2):DIM].argmax(dim=-1, keepdim=True) + int(DIM/2)
    best_path = torch.zeros_like(inputs)
    # This handling of best_path_inds doesn't quite work as intended with 3+ dim input
    best_path_inds = path_inds[...,best_scores_ind].squeeze(-1).transpose(-2, -1)
    best_path.scatter_(dim=-1, index=best_path_inds, value=1.0)
    return best_path

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x)

best_path(torch.rand(10, DIM))

tensor([[3.3185e-01, 3.7064e-01, 7.6091e-01,  ..., 7.2449e-01, 4.0909e-01,
         3.6302e-01],
        [5.4926e-01, 9.0616e-01, 3.4756e-01,  ..., 9.3212e-02, 9.2029e-01,
         7.0653e-02],
        [7.5976e-01, 8.9371e-01, 1.2690e-01,  ..., 9.3196e-01, 7.1579e-04,
         3.8021e-01],
        ...,
        [6.4389e-01, 5.8120e-01, 4.3260e-01,  ..., 4.4259e-02, 4.7809e-01,
         4.1155e-01],
        [7.4787e-01, 6.2935e-01, 2.1095e-02,  ..., 5.0277e-01, 4.2797e-01,
         6.3522e-01],
        [7.8070e-01, 6.5361e-01, 9.7238e-01,  ..., 7.4369e-01, 4.3411e-01,
         6.5070e-01]], dtype=torch.float64)


tensor([[1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [2]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

print("Initially, the values in our tensor do not result in the desired argsort")
print(best_path(x))
print(y_true)

torch.norm(perturbed(best_path, device=device)(x) - y_true, dim=-1)

# FenchelYoungLoss(ranks, device=device)(y_true, x)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.SGD([x], 1.0)
# optim = torch.optim.LBFGS([x], .01)


Initially, the values in our tensor do not result in the desired argsort
tensor([[1., 0., 1.,  ..., 1., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 1., 0., 0.]], dtype=torch.float64)
tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [3]:
%%time 
BASE = 10

for iteration in range(10*BASE+1):
    def closure():
        optim.zero_grad()
    #     criterion = FenchelYoungLoss(ranks, sigma=0.25, device=device)
    #     loss = criterion(x, y_true).sum()
        perturbed_x = perturbed(best_path, num_samples=100, sigma=0.25, noise='gumbel', device=device)(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
        loss.backward()
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        perturbed_x = perturbed(best_path, num_samples=100, sigma=0.25, noise='gumbel', device=device)(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
#         print(perturbed_x)
        print(torch.abs(best_path(x) - y_true).sum().item())
        print(loss.item())
        
print("SGD has run???")

530.0
1.544906080701755
402.0
1.3803906936763828
282.0
1.2114477780853492
220.0
1.0502806231823238
184.0
0.912907864504493
164.0
0.7998588342977688
132.0
0.7062404965477833
118.0
0.633140877323443
102.0
0.5636766537875812
80.0
0.49754726178344844
78.0
0.46107268092121706
SGD has run???
Wall time: 1.35 s


In [4]:
best_path(x).sum(dim=0)

tensor([128., 126.,   2., 121.,   5.,   2.,   0.,  98.,  23.,   5.,   0.,   0.,
          2.,   0.,   0.], dtype=torch.float64)

## Perturbed Fixed Noise

In [5]:
import torch
from perturbations import fixed_noise_perturbed, get_presampled_noises

dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DEPTH = 4
DIM = 2**DEPTH - 1

path_inds = torch.zeros(DEPTH, DIM).long()
path_inds[0] = torch.arange(start=0, end=DIM).long()
path_inds[1] = ((path_inds[0]-1)/2).long()
path_inds[2] = ((path_inds[1]-1)/2).long()
path_inds[3] = ((path_inds[2]-1)/2).long()

def best_path(inputs):
    """Returns the ranks of the input values among the given axis."""
    scores = inputs[..., path_inds].sum(dim=-2).squeeze(dim=-2)
    best_scores_ind = scores[..., int(DIM/2):DIM].argmax(dim=-1, keepdim=True) + int(DIM/2)
    best_path = torch.zeros_like(inputs)
    # This handling of best_path_inds doesn't quite work as intended with 3+ dim input
    best_path_inds = path_inds[...,best_scores_ind].squeeze(-1).transpose(-2, -1)
    best_path.scatter_(dim=-1, index=best_path_inds, value=1.0)
    return best_path

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x)

best_path(torch.rand(10, DIM))

tensor([[0.9520, 0.6853, 0.0169,  ..., 0.3967, 0.9420, 0.0465],
        [0.6383, 0.7843, 0.7522,  ..., 0.3941, 0.5742, 0.5866],
        [0.6394, 0.1132, 0.9993,  ..., 0.0220, 0.2934, 0.2692],
        ...,
        [0.7485, 0.5366, 0.1259,  ..., 0.7921, 0.7855, 0.4668],
        [0.4089, 0.8890, 0.4769,  ..., 0.0082, 0.0758, 0.3279],
        [0.8342, 0.1238, 0.1844,  ..., 0.4754, 0.7336, 0.9265]],
       dtype=torch.float64)


tensor([[1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [6]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

print("Initially, the values in our tensor do not result in the desired argsort")
print(best_path(x))
print(y_true)

presampled_noises = get_presampled_noises(x.shape, num_samples=1000, noise="normal")
torch.norm(
    fixed_noise_perturbed(
        best_path, 
        presampled_noises=presampled_noises, 
        num_samples=1000, 
        sigma=0.25, 
        device=device
    )(x) - y_true, 
    dim=-1
)

# FenchelYoungLoss(ranks, device=device)(y_true, x)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.SGD([x], 1.0)
# optim = torch.optim.LBFGS([x], .1)


Initially, the values in our tensor do not result in the desired argsort
tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.]], dtype=torch.float64)
tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [24]:
%%time 
BASE = 10

N_SAMPLES = 1000

for iteration in range(10*BASE+1):    
    presampled_noises = get_presampled_noises(x.shape, num_samples=N_SAMPLES, noise="normal")
    def closure():
        optim.zero_grad()
    #     criterion = FenchelYoungLoss(ranks, sigma=0.25, device=device)
    #     loss = criterion(x, y_true).sum()
        perturbed_x = fixed_noise_perturbed(
            best_path, 
            presampled_noises=presampled_noises, 
            num_samples=N_SAMPLES, 
            sigma=0.25, 
            device=device
        )(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
        loss.backward()
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        perturbed_x = fixed_noise_perturbed(
            best_path, 
            presampled_noises=presampled_noises, 
            num_samples=N_SAMPLES, 
            sigma=0.25, 
            device=device
        )(x)
        loss = torch.norm(perturbed_x - y_true, dim=-1).mean()
#         print(perturbed_x)
        print(torch.abs(best_path(x) - y_true).sum().item())
        print(loss.item())
        
print("SGD has run???")

IndexError: index 14 is out of bounds for dimension 0 with size 14

## CVXPyLayers - DAG

In [25]:
from cvxpylayers.torch import CvxpyLayer
import torch
import cvxpy as cp

In [26]:
# Define convex optimization model
num_n = 5
num_e = 5
e_out = [
    [0, 1],
    [2, 3],
    [4],
    [],
    [],
]
e_in = [
    [],
    [0],
    [1],
    [2, 4],
    [3],
]

n = cp.Variable(num_n)
e = cp.Variable(num_e)
s_in = cp.Parameter(1)
s_out = cp.Variable(2)
e_hat = cp.Parameter(num_e)

objective = cp.norm2(e-e_hat)
bound_constraints = [e >= 0, e <= 1]
flow_constraints = [
    s_in[0] - e[0] - e[1] == 0,
    e[0] - e[2] - e[3] == 0,
    e[1] - e[4] == 0,
    e[2] + e[4] + s_out[0] == 0,
    e[3] + s_out[1] == 0,
    
]
source_sink_constraints = [
    s_in[0] + s_out[0] + s_out[1] == 0,
]
constraints = bound_constraints + flow_constraints + source_sink_constraints

prob = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)
dag_proj_layer = CvxpyLayer(problem=prob, parameters=[e_hat, s_in], variables=[e, s_out])

In [27]:
# layer(torch.Tensor([1.0]), torch.Tensor([1.0, 0.5]))
path_true = torch.Tensor([1.0, 0.0, 0.0, 1.0, 0.0])
e_arg = torch.Tensor([1.5, 1.0, 1.0, 0.0, 1.0])
e_arg.requires_grad = True
s_in_arg = torch.tensor([1.0])
e_res, s_res = dag_proj_layer(e_arg, s_in_arg)
loss = torch.norm(e_res - path_true)
loss.backward()
e_res

tensor([ 6.2500e-01,  3.7500e-01,  6.2500e-01, -1.2694e-10,  3.7500e-01],
       grad_fn=<_CvxpyLayerFnFnBackward>)

In [28]:
dag_proj_layer(torch.Tensor([e_arg.tolist(), e_arg.tolist()]), torch.Tensor([[1.0], [2.0]]))

(tensor([[ 6.2500e-01,  3.7500e-01,  6.2500e-01, -1.2694e-10,  3.7500e-01],
         [ 1.0000e+00,  1.0000e+00,  9.9997e-01,  3.6081e-05,  1.0000e+00]]),
 tensor([[-1.0000e+00,  1.6819e-10],
         [-2.0000e+00, -3.3548e-05]]))

In [29]:
dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

DIM = 5

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x.shape)
source = torch.ones(x.shape[0], 1).to(**kwargs)
print(source.shape)

torch.Size([128, 5])
torch.Size([128, 1])


In [30]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1.0, 0.0, 0.0, 1.0, 0.0]).to(**kwargs)

print(y_true)

# Initialize an SGD optimizer and do 200 steps
optim = torch.optim.Adam([x], 1.0)
# optim = torch.optim.LBFGS([x], lr=1.0)


tensor([1., 0., 0., 1., 0.], dtype=torch.float64)


In [31]:
%%time 
import time
BASE = 1

fwd = 0
bwd = 0
for iteration in range(10*BASE+1):
    
    def closure():
        optim.zero_grad()
        start = time.time()
        dag_proj, _ = dag_proj_layer(x, source)
        end = time.time()
#         fwd += end - start
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
    #     + torch.norm(dag_proj - x)
    #     + torch.maximum(
    #         torch.norm(x, dim=-1) - torch.Tensor([6]).to(**kwargs), torch.Tensor([0]).to(**kwargs)
    #     ).mean()
        start = time.time()
        loss.backward()
        end = time.time()
#         bwd += end - start  
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        dag_proj, _ = dag_proj_layer(x, source)
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
#         print(perturbed_x)
#         print(dag_proj[:10,:7])
        print(torch.abs(dag_proj - y_true).sum().item())
        print(loss.item(), torch.sum(x, dim=-1).mean().item())
        
print("SGD has run???")
print("TIMES: ", fwd, bwd)

30.00039860204829
0.15925753900266398 1.8224354377598138
30.001746417708592
0.15926332809799687 1.3308338987417847
30.002646526672727
0.15926733196860007 0.9480586989442407
30.002053412962134
0.15926452590498824 0.6352807158939697
30.001823684527313
0.15926320713274444 0.37016834605050386
30.002076394949967
0.15926378584744097 0.14168258562392505
30.002683010649783
0.1592664223676385 -0.058295641796905
30.002731352352562
0.1592665344007294 -0.23461336999044588
30.003660802340036
0.15926983285893184 -0.39127463174818333
30.00314438494598
0.15926810810088876 -0.5311904658046104
30.00214102161243
0.15926432932356316 -0.6565962140890227
SGD has run???
TIMES:  0 0
Wall time: 10.7 s


In [15]:
torch.round(dag_proj).sum(dim=0)

tensor([128.,   0.,  18., 110.,   0.], dtype=torch.float64,
       grad_fn=<SumBackward1>)

## CVXPyLayers - Tree

In [16]:
dtype = torch.double
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

kwargs = {"dtype": dtype, "device": device}

In [17]:
# Define convex optimization model
num_e = 14
num_sinks = 8
e = cp.Variable(num_e)
s_in = cp.Parameter(1)
s_out = cp.Variable(num_sinks)
e_hat = cp.Parameter(num_e)

objective = cp.norm2(e-e_hat)
bound_constraints = [e >= 0, e <= 1]
flow_constraints = [
    e[0] - e[2] - e[3] == 0,
    e[1] - e[4] - e[5] == 0,
    e[2] - e[6] - e[7] == 0,
    e[3] - e[8] - e[9] == 0,
    e[4] - e[10] - e[11] == 0,
    e[5] - e[12] - e[13] == 0,
] + [
    e[6+i] + s_out[i] == 0
    for i in range(num_sinks)
]
source_sink_constraints = [
    s_in[0] + cp.sum(s_out) == 0,
]
constraints = bound_constraints + flow_constraints + source_sink_constraints

prob = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)
dag_proj_layer = CvxpyLayer(problem=prob, parameters=[e_hat, s_in], variables=[e, s_out])

In [18]:
DIM = 14

# We initialize a random tensor
x = torch.rand([128, DIM]).to(**kwargs)
print(x.shape)
source = torch.ones(x.shape[0], 1).to(**kwargs)
print(source.shape)

torch.Size([128, 14])
torch.Size([128, 1])


In [19]:
# Turn its grad on, since we will change this tensor to minimize our loss
x.requires_grad = True
y_true = torch.Tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

# print(best_path(x))
print(y_true)

dag_proj, _ = dag_proj_layer(x, source) 
print(torch.norm(dag_proj - y_true, dim=-1).mean())

# Initialize an SGD optimizer and do 200 steps
# optim = torch.optim.SGD([x], lr=10.0)
optim = torch.optim.LBFGS([x], lr=1.0)

tensor([1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor(1.5538, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [20]:
import time

In [21]:
%%time 
BASE = 1

fwd = 0
bwd = 0
for iteration in range(10*BASE+1):
    
    def closure():
        optim.zero_grad()
        start = time.time()
        dag_proj, _ = dag_proj_layer(x, source)
        end = time.time()
#         fwd += end - start
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
    #     + torch.norm(dag_proj - x)
    #     + torch.maximum(
    #         torch.norm(x, dim=-1) - torch.Tensor([6]).to(**kwargs), torch.Tensor([0]).to(**kwargs)
    #     ).mean()
        start = time.time()
        loss.backward()
        end = time.time()
#         bwd += end - start  
        return loss
    optim.step(closure)
    if iteration % BASE== 0:
        dag_proj, _ = dag_proj_layer(x, source)
        loss = torch.norm(dag_proj - y_true, dim=-1).mean()
#         print(perturbed_x)
#         print(dag_proj[:10,:7])
        print(torch.abs(dag_proj - y_true).sum().item())
        print(loss.item(), torch.sum(x, dim=-1).mean().item())
        
print("SGD has run???")
print("TIMES: ", fwd, bwd)

106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
106.00091032137317
0.520852935131989 6.954080244383761
SGD has run???
TIMES:  0 0
Wall time: 17.4 s


In [22]:
x[:,6]

tensor([3.9357e-01, 9.0108e-02, 2.0178e+01, 2.0237e+01, 2.7333e+01, 7.2641e-01,
        2.6924e+01, 2.0451e+01, 2.1102e+01, 2.2105e+01, 2.1090e+01, 2.4846e+01,
        1.1288e-01, 2.1883e+01, 2.6872e+01, 5.0422e-01, 2.3302e-01, 1.0843e-01,
        1.6690e-01, 2.0878e+01, 3.4643e-01, 1.9815e+01, 6.9619e-02, 1.9880e+01,
        1.9423e+01, 2.4565e+01, 2.1004e+01, 2.1605e+01, 4.8361e-01, 2.1652e+01,
        2.5763e+01, 2.9220e-01, 2.5144e+01, 2.2408e+01, 2.1852e+01, 4.7119e-01,
        2.6044e+01, 2.1563e+01, 2.4398e+01, 1.5851e-01, 2.0480e+01, 3.2645e-03,
        2.3797e+01, 1.8901e+01, 2.5515e+01, 2.2012e+01, 2.0657e+01, 9.4815e-02,
        3.9029e-01, 2.4551e+01, 2.2940e+01, 3.6197e-01, 2.6178e+01, 2.3106e+01,
        1.7896e+01, 3.2030e-01, 2.1907e+01, 1.8311e-01, 1.9685e+01, 2.0427e+01,
        2.0640e-01, 2.0261e+01, 2.7306e-01, 2.2856e+01, 3.1585e-01, 1.0656e-01,
        2.2077e+01, 2.0519e+01, 3.0591e-01, 2.0258e-01, 2.1737e+01, 2.2918e+01,
        2.1432e+01, 2.0719e+01, 2.2451e+

In [23]:
dag_proj[:,[0,2,6]]

tensor([[ 1.0000e+00,  1.0000e+00,  6.6653e-11],
        [ 1.0000e+00,  1.6440e-10,  4.2515e-11],
        [ 1.0000e+00,  9.9999e-01,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  3.0006e-12, -7.8576e-12],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00, -3.0783e-11],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00,  5.2019e-06,  5.5726e-06],
        [ 9.9999e-01, -1.5839e-05, -6.6986e-06],
        [ 1.0000e+00,  1.0000e+00,  1.8789e-09],
        [ 1.0000e+00,  1.0000e+00,  1.8043e-11],
        [ 1.0000e+00,  1.0000e+00,  1.0000e+00],
        [ 1.0000e+00